In [0]:
import typing as T
from datetime import datetime


In [0]:
# General Functions

In [0]:

def log_function_execution(func: T.Callable[..., T.Any]):
    def wrapper(*args, **kwargs):
        print(f"{func.__name__}-inicio-{datetime.now()}")
        return_value = func(*args, **kwargs)
        print(f"""{func.__name__}-fim-{datetime.now()}
              """)
        return return_value
    return wrapper

In [0]:
@log_function_execution
def stop_all_streams():
    for stream_query in spark.streams.active:
        try:
            stream_query.stop()
            print(f'O stream {stream_query.name} fui finalizado com sucesso.')
        except RuntimeError:
            pass

def clean_up_tables(database="default"):
    try: 
        spark.sql(f"USE {database}")
        for table in spark.sql('show tables').collect():
            if table['isTemporary'] is not True:
                full_table_name = f"{table['database']}.{table['tableName']}"
                spark.sql(f"DROP TABLE {full_table_name}")
                print(f"A tabela [{full_table_name}] foi excluída com sucesso.")
    except:
        print(f"O database [{database}] não existe.")

@log_function_execution
def clean_up_landing_dir():
    dbutils.fs.rm("dbfs:/FileStore/landing/",True)
    print(f"Todos os arquivos e diretórios dentro de 'dbfs:/FileStore/landing/' foram excluidos com sucesso.")

@log_function_execution
def clean_up_database(database='default'):
    try:
        clean_up_tables(database)
        spark.sql(f'DROP DATABASE {database}')
        print(f'O database [{database}] foi excluido com sucesso.')
    except:
        pass